In [101]:
import copy
import pandas as pd
import random
import re
import sys

inputFile = pd.read_csv(u"Управление анкетой 2.csv", header=-1, sep='\t')
inputFile = inputFile.fillna('')
lengthOfInputFile = inputFile.shape[0]
widthOfInputFile = inputFile.shape[1]
N = 1000    # Количество опрашиваемых

In [102]:
inputFile

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,#1,,,,,,,,,,,,,,
1,Из какой страны вы приехали на работу в Россию?,,,,,,,,,,,,,,
2,Узбекистан,Армения,XXX,,,,,,,,,,,,
3,0.6,0.3,0.1,,,,,,,,,,,,
4,#2,,,,,,,,,,,,,,
5,Кто Вы по национальности?,,,,,,,,,,,,,,
6,Узбек,Армянин,Киргиз,Азербайджанец,Таджик,Молдованин,Русский,Грузин,Другое (напишите),,,,,,
7,0.6,0.3,0.1,0,0,0,0,0,0,,,,,,
8,#3,,,,,,,,,,,,,,
9,Как долго Вы проживаете в России?,,,,,,,,,,,,,,


In [103]:
inputFile[0][3]

'0.6'

# Вспомогательные функции

## Добавление распределения

In [104]:
def addDistribution(distributionsDict, inputFile, iVariants, iValues, title, key):
    j = 0
    distributionsDict[title][key] = {}
    while True:
        #print 'строка ' + str(iValues) + '\tстолбец ' + str(j)
        if j >= inputFile.shape[1]:
            print 'Край таблицы'
            break
        if inputFile[j][iVariants] == '':
            print 'Кончились варианты'
            break
        if inputFile[j][iValues] == '':
            print 'Отсутствует вероятность для ' + inputFile[j][iVariants] + ' в строке ' + str(iValues)
            return -1
        distributionsDict[title][key][inputFile[j][iVariants]] = inputFile[j][iValues]
        j += 1
    
    # Проверка
    mySum = 0
    for item in distributionsDict[title][key]:
        mySum += float(distributionsDict[title][key][item])
    if mySum > 1:
        print 'Неправильно указаны вероятности для вопроса ' + str(title) + ' в строке ' + str(iValues)
        return -1
    
    print pd.DataFrame(distributionsDict[title])
    return 0

## Добавление ответа

In [105]:
def addAnswer(answersDict, title, distribution):
    answer = random.random()
    #print answer
    i = 1
    mySum = 0
    for item in distribution:
        #print item
        #print distribution[item]
        mySum += float(distribution[item])
        #print mySum
        if answer < mySum:
            #print 'answer ' + str(answer) + '\t<\tmySum ' + str(mySum)
            answer = i
            #print 'Final answer:\t' + str(answer)
            break
        #else:
            #print 'answer ' + str(answer) + '\t>\tmySum ' + str(mySum)
        i += 1
    #print answer
    answersDict[title].append(answer)
    return 0

# Основное тело программы
## Создаём ключи словаря

In [106]:
questionsDict = {}
for i in range(1, 23):
    questionsDict['q' + str(i)] = {}

## Заполняем правила

In [107]:
distributionsDict = copy.deepcopy(questionsDict)
questionNumber = 0
for i in range(lengthOfInputFile):  # Идём по каждой строке исходной таблицы
    firstCell = inputFile[0][i]
    if firstCell.lstrip('#') == str(questionNumber + 1):    # Нашли что-то похожее на вопрос?
        questionNumber += 1
        
        if questionNumber == 21:
            continue
        
        # Здесь надо будет научиться определять тип вопроса
        
        # Выбор одного из вариантов
        j = 0
        EndOfQuestion = False
        while not EndOfQuestion:
            errorCode = 0
            
            if i + j >= lengthOfInputFile:    # Обработка конца таблицы
                EndOfQuestion = True
                continue
                
            firstCell = inputFile[0][i + j]    # Новый вопрос знаменуется строкой с решёткой в начале
            if firstCell.lstrip('#') == str(questionNumber + 1):
                EndOfQuestion = True
                continue
            
            if j == 2:    # Распределение по умолчанию
                errorCode = addDistribution(distributionsDict, inputFile, i + j, i + j + 1, 'q' + str(questionNumber), 'Default')
            
            if firstCell.find('&&&') != -1: # Распределение с логическим условием
                k = 1
                key = ['&&&']
                while inputFile[k][i + j] != '':
                    key.append( (inputFile[k][i + j], inputFile[k + 1][i + j]) ) 
                    if k+1 == widthOfInputFile:
                        break
                    k += 2
                key = tuple(key)
                
                errorCode = addDistribution(distributionsDict, inputFile, i + 2, i + j + 1, 'q' + str(questionNumber), key)
            
            if ( errorCode == -1 ):
                print 'Поправьте распределение!'
                sys.exit()
                
            #print questionNumber
                
               
            j += 1
        
        questionsDict['q' + str(questionNumber)]

Кончились варианты
           Default
XXX            0.1
Армения        0.3
Узбекистан     0.6
Кончились варианты
                  Default
Азербайджанец           0
Армянин               0.3
Грузин                  0
Другое (напишите)       0
Киргиз                0.1
Молдованин              0
Русский                 0
Таджик                  0
Узбек                 0.6
Кончились варианты
                        Default
Менее 6 месяцев             0.6
Около года                  0.3
Сколько лет (напишите):     0.1
Кончились варианты
                        Default (&&&, (2, Узбек))
Менее 6 месяцев             0.6               0.6
Около года                  0.3               0.3
Сколько лет (напишите):     0.1               0.1
Кончились варианты
                                               Default
В жилых помещениях у работодателя                  0.1
В нежилом помещении (подвале, чердаке, бараке)       0
В общежитии                                        0.3
На съёмной квартире  

In [108]:
pd.DataFrame(distributionsDict)

,q1,q10,q11,q12,q13,q14,q15,q16,q17,q18,...,q20,q21,q22,q3,q4,q5,q6,q7,q8,q9
Default,"{u'XXX': u'0.1', u'Узбекистан': u'0.6', u'Арме...","{u'Примерно поровну как на родном, так и на ру...","{u'Взятки в полиции/органах власти.': u'0.3', ...","{u'Да': u'0.6', u'Нет': u'0.3'}","{u'Да': u'0.6', u'Нет': u'0.4'}","{u'К некоммерческим организациям, работающим с...","{u'В диаспоре': u'0', u'Интернет.': u'0.6', u'...","{u'В диаспоре': u'0', u'Интернет.': u'0.6', u'...","{u'Да': u'0.6', u'Нет': u'0.4'}","{u'Скорее плохо': u'0.1', u'Хорошо': u'0.6', u...",...,"{u'Скорее плохо': u'0.1', u'Хорошо': u'0.6', u...",NaN,"{u'Ученая степень.': u'0', u'Высшее.': u'0', u...","{u'Менее 6 месяцев': u'0.6', u'Сколько лет (на...","{u'На съёмной квартире': u'0.6', u'В общежитии...","{u'Не знаю, что буду делать': u'0', u'Эмигриру...","{u'Неплохо говорю на русском, но имею проблемы...",{u'С людьми своей национальности и мигрантами ...,{u'С людьми своей национальности и мигрантами ...,{u'Могут жить и работать/учиться в России и бе...
"(&&&, (2, Узбек))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"{u'Менее 6 месяцев': u'0.6', u'Сколько лет (на...",NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
distributionsDict

{'q1': {'Default': {'XXX': '0.1',
   '\xd0\x90\xd1\x80\xd0\xbc\xd0\xb5\xd0\xbd\xd0\xb8\xd1\x8f': '0.3',
   '\xd0\xa3\xd0\xb7\xd0\xb1\xd0\xb5\xd0\xba\xd0\xb8\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd': '0.6'}},
 'q10': {'Default': {'\xd0\x92 \xd0\xbe\xd1\x81\xd0\xbd\xd0\xbe\xd0\xb2\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xbd\xd0\xb0 \xd1\x80\xd0\xbe\xd0\xb4\xd0\xbd\xd0\xbe\xd0\xbc \xd1\x8f\xd0\xb7\xd1\x8b\xd0\xba\xd0\xb5 (\xd0\xbd\xd0\xb5 \xd1\x80\xd1\x83\xd1\x81\xd1\x81\xd0\xba\xd0\xbe\xd0\xbc)': '0.3',
   '\xd0\x92 \xd0\xbe\xd1\x81\xd0\xbd\xd0\xbe\xd0\xb2\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xbd\xd0\xb0 \xd1\x80\xd1\x83\xd1\x81\xd1\x81\xd0\xba\xd0\xbe\xd0\xbc': '0',
   '\xd0\x9f\xd1\x80\xd0\xb8\xd0\xbc\xd0\xb5\xd1\x80\xd0\xbd\xd0\xbe \xd0\xbf\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb2\xd0\xbd\xd1\x83 \xd0\xba\xd0\xb0\xd0\xba \xd0\xbd\xd0\xb0 \xd1\x80\xd0\xbe\xd0\xb4\xd0\xbd\xd0\xbe\xd0\xbc, \xd1\x82\xd0\xb0\xd0\xba \xd0\xb8 \xd0\xbd\xd0\xb0 \xd1\x80\xd1\x83\xd1\x81\xd1\x81\xd0\xba\xd0\xbe\xd0\xbc': '0.1',
   '\xd0\xa2\xd0\x

# Формируем заголовки таблицы ответов

In [110]:
answersDict = copy.deepcopy(questionsDict)
for i in range(1, 23):
    title = 'q' + str(i)
    answersDict[title] = []

In [111]:
answersDict['q1']

[]

# Бегаем по опрашиваемым

In [112]:
for i in range(N):
    questionNumber = 0
    for j in range(1, 23):
        
        title = 'q' + str(j)
        
        
        if j == 21:    #заглушка
            answersDict[title].append(0)
            continue
            
        key = 'Default'
        for distribution in distributionsDict[title]:
            if distribution[0] == '&&&':
                if answersDict['q'+str(distribution[1][0])] == str(distribution[1][1]):
                    key = distribution
        
        addAnswer(answersDict, title, distributionsDict[title][key])

        

    
    

In [113]:
result = pd.DataFrame(answersDict)

In [114]:
#result.columns

In [115]:
#sorted(sorted(result.columns), key = lambda myStr: len(myStr))

In [116]:
result = result.reindex(sorted(sorted(result.columns), key = lambda myStr: len(myStr)), axis=1)

In [117]:
result.to_csv('output.csv', sep='\t', header=True, index=False)